In [1]:
import warnings
warnings.filterwarnings('ignore')

from scripts import utils
from pprint import pprint

import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["pdfs/eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [3]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

## Window-sentence retrieval setup

In [4]:
from llama_index.node_parser import SentenceWindowNodeParser

node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [5]:
text = "hello. how are you? I am fine!  "

nodes = node_parser.get_nodes_from_documents([Document(text=text)])

In [6]:
print([x.text for x in nodes])

['hello. ', 'how are you? ', 'I am fine!  ']


In [7]:
print(nodes[1].metadata["window"])

hello.  how are you?  I am fine!  


In [18]:
# node_parser = SentenceWindowNodeParser.from_defaults(
#     window_size=1,
#     window_metadata_key="window",
#     original_text_metadata_key="original_text",
# )

# text = "hello. foo bar. cat dog. mouse"

# nodes = node_parser.get_nodes_from_documents([Document(text=text)])

# print([x.text for x in nodes])

# print("\n")

# print(nodes[3].metadata["window"])

### Building the index

In [8]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [11]:
from llama_index import ServiceContext

sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    # embed_model="local:BAAI/bge-large-en-v1.5"
    node_parser=node_parser,
)

In [12]:
from llama_index import VectorStoreIndex

sentence_index = VectorStoreIndex.from_documents(
    [document], service_context=sentence_context
)

In [ ]:
sentence_index.storage_context.persist(persist_dir="./sentence_index")


In [13]:
# This block of code is optional to check
# if an index file exist, then it will load it
# if not, it will rebuild it

import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage

if not os.path.exists("./sentence_index"):
    sentence_index = VectorStoreIndex.from_documents(
        [document], service_context=sentence_context
    )

    sentence_index.storage_context.persist(persist_dir="./sentence_index")
else:
    sentence_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./sentence_index"),
        service_context=sentence_context,
    )

### Building the postprocessor

In [14]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

postproc = MetadataReplacementPostProcessor(target_metadata_key="window")

In [ ]:
print([x.text for x in nodes])

['hello. ', 'how are you? ', 'I am fine!  ']


In [15]:
from llama_index.schema import NodeWithScore
from copy import deepcopy

# replicate similarity search result
scored_nodes = [NodeWithScore(node=x, score=1.0) for x in nodes] 

nodes_old = [deepcopy(n) for n in nodes]

In [ ]:
print([x.text for x in nodes_old])

['hello. ', 'how are you? ', 'I am fine!  ']


In [ ]:
nodes_old[1].text

'how are you? '

In [ ]:
replaced_nodes = postproc.postprocess_nodes(scored_nodes)

In [ ]:
print(replaced_nodes[1].text)

hello.  how are you?  I am fine!  


### Adding a Reranker

In [16]:
from llama_index.indices.postprocessor import SentenceTransformerRerank

# BAAI/bge-reranker-base
# link: https://huggingface.co/BAAI/bge-reranker-base
rerank = SentenceTransformerRerank(
    top_n=2, model="BAAI/bge-reranker-base"
)

In [17]:
from llama_index import QueryBundle
from llama_index.schema import TextNode, NodeWithScore

query = QueryBundle("I want a dog.")

scored_nodes = [
    NodeWithScore(node=TextNode(text="This is a cat"), score=0.6),
    NodeWithScore(node=TextNode(text="This is a dog"), score=0.4),
    NodeWithScore(node=TextNode(text="A dog is a domestic animal"), score=0.4),
    NodeWithScore(node=TextNode(text="This is a lion"), score=0.4),
]

In [18]:
reranked_nodes = rerank.postprocess_nodes(
    scored_nodes, query_bundle=query
)

print([(x.text, x.score) for x in reranked_nodes])

[('This is a dog', 0.91827375), ('A dog is a domestic animal', 0.76897246)]


### Running the Query Engine

In [19]:
sentence_window_engine = sentence_index.as_query_engine(
    similarity_top_k=6, node_postprocessors=[postproc, rerank]
) 

In [20]:
window_response = sentence_window_engine.query(
    "What are the keys to building a career in AI?"
)

In [21]:
from llama_index.response.notebook_utils import display_response

display_response(window_response)

**`Final Response:`** The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community.

### Putting it all together

In [22]:
import os
from typing import List

from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage


def build_sentence_window_index(
    documents: List[Document],
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index: VectorStoreIndex, similarity_top_k=6, rerank_top_n=2
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

In [23]:
from llama_index.llms import OpenAI

index = build_sentence_window_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./sentence_index",
)


In [24]:
query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)

In [25]:
pprint(
    query_engine.query(
        "What are the main skills needed to build a career in AI"
    ).response
)

('The main skills needed to build a career in AI include learning foundational '
 'technical skills, working on projects, and finding a job. Additionally, '
 'having a supportive community can also be beneficial in this career path.')


## TruLens Evaluation

In [26]:
eval_questions = []
with open("texts/generated_questions.text", "r") as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [27]:
from trulens_eval import Tru

tru = Tru(database_file="sentence_window.sqlite")

# tru.reset_database()

🦑 Tru initialized with db url sqlite:///sentence_window.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [28]:
from trulens_eval import TruLlama
from llama_index.core.base_query_engine import BaseQueryEngine
from scripts.utils import get_prebuilt_trulens_recorder


def run_evals(
    eval_questions: List[str], tru_recorder: TruLlama, query_engine: BaseQueryEngine
) -> None:
    for question in eval_questions:
        with tru_recorder as _:
            query_engine.query(question)


def get_evals(window_size: int) -> None:
    sentence_index = build_sentence_window_index(
        documents,
        llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
        embed_model="local:BAAI/bge-small-en-v1.5",
        sentence_window_size=1,
        save_dir=f"sentence_index_{window_size}",
    )

    sentence_window_engine = get_sentence_window_query_engine(sentence_index)

    tru_recorder = get_prebuilt_trulens_recorder(
        sentence_window_engine, app_id=f"sentence window engine {window_size}"
    )

    run_evals(eval_questions, tru_recorder, sentence_window_engine)

### Sentence window size = 1

In [29]:
get_evals(1)

In [33]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.161:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [36]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
sentence window engine 1,0.4,0.7,1.0,19.0,0.000937


### Sentence window size = 3

In [37]:
get_evals(3)

In [40]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
sentence window engine 3,0.9625,0.85,1.0,19.0,0.001371
sentence window engine 1,0.4000,0.70,1.0,19.0,0.000937


### Sentence window size = 5

In [41]:
get_evals(5)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f2848f86dd0 is calling an instrumented method <function BaseQueryEngine.query at 0x7f28a1e654e0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f28501130d0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f2848f86dd0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f288f123560>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f28501130d0) using this function.
A new object of type <class 'llama_index.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> at 0x7f28407184d0 is calling an instrumented method <function BaseRetriever.retrieve at 0x7f2891f59080>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other obj

In [43]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
sentence window engine 3,0.962500,0.85,1.0,19.0,0.001371
sentence window engine 1,0.400000,0.70,1.0,19.0,0.000937
sentence window engine 5,0.345455,0.70,1.0,19.0,0.001047
